In [12]:
import os
import glob
from IPython.display import Image as IPImage
import pandas as pd             # Pandas
import numpy as np              # NumPy
import matplotlib.pyplot as plt # Matplotlib
import seaborn as sns           # Seaborn
from PIL import Image           # Pillow

# TensorFlow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model

# Keras
from keras.layers import Flatten, Dense, Activation, Dropout
from keras import models, optimizers
from keras.models import Sequential
from keras.constraints import MaxNorm
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications import DenseNet121
from keras.applications.densenet import DenseNet121, preprocess_input

from torchvision import transforms

from torch.utils.data import DataLoader, random_split, Dataset, random_split

# scikit-learn
from sklearn.model_selection import train_test_split

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Converts to [0, 1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225]),
])

In [14]:
# read train file 
class BreakHisDataset(Dataset): # Subclass Dataset, which is required for using DataLoader
    def __init__(self, csv_path, transform=None):
        self.df = pd.read_csv(csv_path)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filepath']
        label = self.df.loc[idx, 'label']
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

dataset_train = BreakHisDataset(csv_path="../data/augmented_train_dataset.csv", transform=transform) # Load the data
dataset_test = BreakHisDataset(csv_path="../data/new_test.csv", transform=transform)

# Create iterable data loaders

train = DataLoader(dataset_train, batch_size=16, shuffle=True)
test = DataLoader(dataset_test, batch_size=16, shuffle=False) # shuffle=False for consistent evaluation

In [22]:
# Check a batch from the train DataLoader
for images, labels in train:
    # Print labels
    print("Labels in this batch:", labels)
    
    # Print the shape of the images in the batch (should be [batch_size, C, H, W])
    print("Shape of images:", images.shape)  # This will give you (batch_size, 3, 224, 224)

    # Optionally, print the size of the first image in the batch (or any image)
    print("Shape of the first image:", images[0].shape)  # (3, 224, 224)
    
    # Check if labels match the expected format
    print("First label:", labels[0].item())  # Get the first label as an integer
    
    break  # Stop after the first batch (remove this if you want to check multiple batches)

# Check a batch from the test DataLoader (same process as for train)
for images, labels in test:
    # Print labels
    print("Labels in this batch:", labels)
    
    # Print the shape of the images in the batch
    print("Shape of images:", images.shape)  # (batch_size, 3, 224, 224)
    
    # Check the size of the first image in the batch
    print("Shape of the first image:", images[0].shape)
    
    # First label
    print("First label:", labels[0].item())  # Get the first label
    break  # Stop after the first batch


FileNotFoundError: [Errno 2] No such file or directory: '../train_aug/original_5941.png'